FREED++, VAE долго "поднимать", нужно ковырять докер, ставить другую версию CUDA + у меня потенциально мало видеопамяти для файнтюна, поэтому ...

=> Выбор пал на вычислительно простой генетический алгоритм, хоть он и даёт меньшее качество и не отображает в латентное признаковое пространство

In [1]:
import random
import numpy as np
import pandas as pd
from tqdm.auto import tqdm

from rdkit import Chem
from rdkit.Chem import BRICS, QED
from rdkit.Chem.Lipinski import NumHDonors, NumHAcceptors
from rdkit.Chem.Descriptors import MolLogP
from rdkit.Chem import rdMolDescriptors
from rdkit.Contrib.SA_Score import sascorer

from catboost import CatBoostRegressor
from sklearn.preprocessing import StandardScaler
from joblib import load, Parallel, delayed


# reproducibility
SEED = 42
random.seed(SEED)
np.random.seed(SEED)

o:\Sync\_\DataCon_DD2025\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Подготовка пула фрагментов из множества известных молекул

In [2]:
# Загрузка CSV с дескрипторами
df_full = pd.read_csv('../../data/target_cox2_IC50__descriptors_with_ECFP6.csv')

actives = [
    s for s in df_full['Smiles']
    if Chem.MolFromSmiles(s)
]
print("Исходных валидных SMILES:", len(actives))


# Декомпозиция через BRICS → собираем уникальные фрагменты
frag_smiles = set()
for smi in actives:
    mol = Chem.MolFromSmiles(smi)
    frag_smiles |= BRICS.BRICSDecompose(mol)

# Конвертация SMILES-фрагментов в Mol, отбрасываем невалидные
frag_mols = [Chem.MolFromSmiles(s) for s in frag_smiles]
frag_mols = [m for m in frag_mols if m]
print(f"Extracted {len(frag_mols)} valid BRICS fragments")

C:\Users\MRVen\AppData\Local\Temp\ipykernel_22744\3271841608.py:2: DtypeWarning: Columns (1487,1488,1510,1511,1512,1513,1514,1545,1547,1548,1557,1558,1559,1560,1561,1562,1578,1581,1582,1585) have mixed types. Specify dtype option on import or set low_memory=False.
  df_full = pd.read_csv('../../data/target_cox2_IC50__descriptors_with_ECFP6.csv')


Исходных валидных SMILES: 3967
Extracted 1742 valid BRICS fragments


# Генерация кандидатов BRICS+GA

In [3]:
# Параметры генерации
N = 42                   # ← Желаемое число уникальных кандидатов
K = 3 * N                # Количество попыток с запасом

# Пул фрагментов: frag_mols должен быть списком RDKit Mol-объектов
idxs = list(range(len(frag_mols)))

def gen_one(_):
    i, j, k = random.sample(idxs, 3)
    for mol in BRICS.BRICSBuild([frag_mols[i], frag_mols[j], frag_mols[k]]):
        smi = Chem.MolToSmiles(mol, isomericSmiles=True)
        if Chem.MolFromSmiles(smi):
            return smi
    return None

# Параллельная генерация K кандидатов
results = Parallel(n_jobs=-1, prefer="threads")(
    delayed(gen_one)(_) for _ in tqdm(range(K), desc="Generating BRICS+GA")
)

# Убираем None, сохраняем порядок, убираем дубликаты
unique_ordered = list(dict.fromkeys([s for s in results if s]))

# Берём первые N уникальных SMILES
candidates = unique_ordered[:N]

print("BRICS+GA generated:", len(candidates))

Generating BRICS+GA: 100%|██████████| 126/126 [10:11<00:00,  4.85s/it]


BRICS+GA generated: 42


# Валидация и фильтрация сгенерированных SMILES, сохранение для расчёта дескрипторов и fingerprint`ов скриптом из мини-таска 2

In [24]:
original_smiles = set(actives)
valid_smiles    = {
    Chem.MolToSmiles(Chem.MolFromSmiles(smi))
    for smi in candidates
    if Chem.MolFromSmiles(smi)
}
new_only = sorted(valid_smiles - original_smiles)
print("Новых уникальных SMILES:", len(new_only))

# Сохраняем SMILES для последующего фичерайзинга (Task 2)
path = '../../data/new_molecules__target_cox2_pIC50__smiles.csv'
pd.DataFrame({'Smiles': new_only}).to_csv(path, index=False)
print("Saved:", path)

Новых уникальных SMILES: 42
Saved: ../../data/new_molecules__target_cox2_pIC50__smiles.csv


# Загрузка предрасчитанных дескрипторов для новых молекул
_*расчёты производятся скриптом из 2 мини-таска_

In [39]:
# тут есть столбец 'Smiles' и все дескрипторы/FP, на которых обучалась модель
df_new = pd.read_csv('../../data/new_molecules__target_cox2_pIC50__descriptors_with_ECFP6.csv')
print("Original shape:", df_new.shape)

# Выравниваем фичи
non_feat = ['Smiles', 'Molecule ChEMBL ID', 'pIC50']
df_full_old  = pd.read_csv('../../data/target_cox2_IC50__descriptors_with_ECFP6.csv')
feature_names = [c for c in df_full_old.columns if c not in non_feat]
print("В df_new_feat:", len(df_new.columns), "колонок")
print("Пропущенные:", set(feature_names) - set(df_new.columns))
print("Лишние:   ", set(df_new.columns) - set(feature_names))

# Выбираем и переупорядочиваем колонки, заполняя отсутствующие нулями
df_aligned = df_new.reindex(columns=feature_names, fill_value=0)

# Проверяем форму
print("После выравнивания:", df_aligned.shape)  # должно быть (n_samples, 1718)

# Колонки, которые не участвуют в предсказании модели
# df_new_feat = df_aligned.drop(columns=['Smiles'])

#Приводим всё к float64, нечисловые значения → NaN
df_new_feat = df_aligned.apply(pd.to_numeric, errors='coerce')

# Сразу убираем признаки, где хоть в одном образце NaN
df_new_feat = df_new_feat.dropna(axis=1, how='any')
print("After dropna columns:", df_new_feat.shape)
df_new_feat

Original shape: (42, 388)


C:\Users\MRVen\AppData\Local\Temp\ipykernel_22744\2474162769.py:7: DtypeWarning: Columns (1487,1488,1510,1511,1512,1513,1514,1545,1547,1548,1557,1558,1559,1560,1561,1562,1578,1581,1582,1585) have mixed types. Specify dtype option on import or set low_memory=False.
  df_full_old  = pd.read_csv('../../data/target_cox2_IC50__descriptors_with_ECFP6.csv')


В df_new_feat: 388 колонок
Пропущенные: {'ECFP4_486', 'ECFP6_783', 'ECFP6_183', 'ECFP6_955', 'ECFP4_764', 'GATS2d', 'ECFP4_382', 'MACCS_115', 'ECFP4_443', 'ECFP4_23', 'ECFP6_238', 'ECFP4_778', 'ECFP4_494', 'ECFP6_336', 'ATSC6s', 'PubchemFP768', 'ECFP4_537', 'ECFP4_905', 'ECFP6_299', 'ECFP4_563', 'ATSC3i', 'ECFP4_106', 'fr_ether', 'ECFP6_176', 'ECFP6_493', 'ATSC7p', 'PubchemFP1', 'ECFP6_547', 'PubchemFP182', 'n11FRing', 'PubchemFP580', 'ECFP4_148', 'ECFP4_580', 'ECFP4_133', 'ECFP4_673', 'ECFP4_741', 'ECFP6_762', 'MACCS_142', 'ECFP4_802', 'ECFP4_851', 'ECFP4_803', 'PubchemFP737', 'ECFP4_414', 'ECFP4_877', 'PubchemFP494', 'fr_Ar_NH', 'ECFP6_590', 'ECFP4_843', 'ECFP4_500', 'ECFP4_273', 'ECFP4_629', 'fr_NH2', 'ECFP4_341', 'ECFP4_648', 'ECFP6_17', 'ECFP4_368', 'ECFP4_612', 'PubchemFP750', 'ECFP6_652', 'ECFP6_602', 'GATS8i', 'ECFP4_887', 'fr_nitro', 'ECFP4_979', 'ECFP4_889', 'ECFP6_746', 'ECFP6_535', 'ECFP6_0', 'ECFP4_818', 'ECFP4_208', 'ECFP4_798', 'ECFP4_380', 'ECFP4_116', 'ECFP4_575', 'ECF

,MaxAbsEStateIndex,MinAbsEStateIndex,MinEStateIndex,qed,SPS,MaxPartialCharge,MinPartialCharge,FpDensityMorgan1,AvgIpc,Ipc,...,PubchemFP768,PubchemFP770,PubchemFP771,PubchemFP772,PubchemFP839,PubchemFP840,PubchemFP845,PubchemFP860,PubchemFP863,PubchemFP868
0,12.998950,0,-1.377580,0.269472,13.794118,0,-0.477762,0.852941,2.821245,2.375601e+07,...,0,0,0,0,0,0,0,0,0,0
1,13.126592,0,-9.970684,0.245493,18.531250,0,-0.477718,1.343750,3.035174,5.108939e+06,...,0,0,0,0,0,0,0,0,0,0
2,12.885088,0,-0.738483,0.394922,38.333333,0,-0.507909,1.296296,3.040238,1.211626e+06,...,0,0,0,0,0,0,0,0,0,0
3,12.145024,0,-0.232587,0.784360,16.791667,0,-0.294749,1.041667,3.073678,6.280631e+05,...,0,0,0,0,0,0,0,0,0,0
4,10.123981,0,-1.194537,0.552910,12.200000,0,-0.477871,0.900000,1.840619,1.067559e+02,...,0,0,0,0,0,0,0,0,0,0
5,4.895070,0,0.193122,0.463681,21.531250,0,-0.354446,1.093750,3.460425,4.549318e+07,...,0,0,0,0,0,0,0,0,0,0
6,2.318611,0,1.277990,0.387226,11.000000,0,-0.088539,0.800000,1.967154,1.750767e+02,...,0,0,0,0,0,0,0,0,0,0
7,6.262946,0,0.946370,0.350520,11.000000,0,-0.084031,0.761905,2.464594,4.714769e+04,...,0,0,0,0,0,0,0,0,0,0
8,11.036874,0,-0.263781,0.656922,9.583333,0,-0.285701,1.083333,2.038174,3.587187e+02,...,0,0,0,0,0,0,0,0,0,0
9,6.423192,0,0.200286,0.468886,50.363636,0,-0.374592,0.666667,3.145713,6.353149e+07,...,0,0,0,0,0,0,0,0,0,0


# Подготовка X и предсказание pIC50

In [42]:
# Загружаем StandardScaler и CatBoost-модель, обученные в Task 3
scaler = load('../scaler.pkl')
model  = CatBoostRegressor()
model.load_model('../models/catboost_cox2_model.cbm')

n_feats = scaler.scale_.shape[0]  
print("Scaler ожидает признаков:", n_feats)


# Масштабируем и предсказываем
X_new = df_new_feat.values
X_scaled = scaler.transform(X_new)
df_new_feat['pIC50_pred'] = model.predict(X_scaled)
print("Предсказано pIC50_pred для", len(df_new_feat), "молекул")

Scaler ожидает признаков: 1718


ValueError: X has 1739 features, but StandardScaler is expecting 1718 features as input.

# Расчёт QED, SA Score, токсофоров и Lipinski

In [ ]:
# SMARTS паттерны токсофоров
tox_smarts = ['[$([NX3](=O)=O)]', '[#6]~[#8]=O']
tox_patts  = [Chem.MolFromSmarts(s) for s in tox_smarts]

# Создаём Mol-объекты
df_aligned['Mol'] = df_aligned['Smiles'].apply(Chem.MolFromSmiles)

# Рассчитываем свойства
df_aligned['QED']  = df_aligned['Mol'].apply(QED.qed)
df_new['SA']   = df_new['Mol'].apply(sascorer.calculateScore)
df_new['Tox']  = df_new['Mol'].apply(
    lambda m: any(m.HasSubstructMatch(t) for t in tox_patts)
).astype(int)

df_new['HDonors']    = df_new['Mol'].apply(NumHDonors)
df_new['HAcceptors'] = df_new['Mol'].apply(NumHAcceptors)
df_new['LogP']       = df_new['Mol'].apply(MolLogP)
df_new['MW']         = df_new['Mol'].apply(rdMolDescriptors.CalcExactMolWt)

df_new['Lipinski_violations'] = (
    (df_new['HDonors']    > 5).astype(int) +
    (df_new['HAcceptors'] > 10).astype(int) +
    (df_new['MW']         > 500).astype(int) +
    (df_new['LogP']       > 5).astype(int)
)

print("Вычислены QED, SA, Tox и Lipinski-нарушения")

# Отбор финальных хитов и сохранение

In [ ]:
# Фильтрация по критериям
hits = df_new.query(
    'pIC50_pred > 6.0 and QED > 0.7 and 2 < SA < 6 '
    'and Tox == 0 and Lipinski_violations <= 1'
).copy()
print("Отобрано хитов:", len(hits))

# Формируем итоговый DataFrame
out = hits[['Smiles','pIC50_pred','QED','SA','Tox','Lipinski_violations']].rename(
    columns={'Smiles':'SMILES', 'pIC50_pred':'pIC50'}
)
out['Comment'] = out.apply(
    lambda r: f"pIC50={r.pIC50:.2f}, QED={r.QED:.2f}, SA={r.SA:.2f}", axis=1
)

# Сохраняем финальный CSV
out.to_csv('selected_hits.csv', index=False)

# Визуализация результатов

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

sns.histplot(df_new['pIC50_pred'], bins=30, kde=True)
plt.title("Predicted pIC50 Distribution")
plt.xlabel("pIC50"); plt.ylabel("Count")
plt.show()

sns.scatterplot(x='SA', y='QED', data=hits)
plt.title("SA vs QED for Selected Hits")
plt.xlabel("SA Score"); plt.ylabel("QED")
plt.show()